In [3]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

     |████████████████████████████████| 4.3MB 2.5MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [1]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_carprice/'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_carprice


In [2]:
ls

data/                      day_3_simple_model.ipynb  LICENSE
day_2_visualization.ipynb  day_4_moremodels.ipynb    README.md


In [4]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

from hyperopt import hp, fmin, tpe, STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [7]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

## Feature engineering

In [8]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue
  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = df[feat].factorize()[0]

cat_feats = [ x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [ x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x).lower()=='none' else int(x))
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x).lower()=='none' else int(x.split(' ')[0]))
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x).lower()=='none' else int(x.split('cm')[0].replace(' ', '')))
feats = ['param_rok-produkcji','param_stan__cat','param_napęd__cat','param_marka-pojazdu__cat','param_skrzynia-biegów__cat','param_moc','param_faktura-vat__cat','param_typ__cat','param_wersja__cat','seller_name__cat','param_pojemność-skokowa','param_model-pojazdu__cat','param_kod-silnika__cat','feature_kamera-cofania__cat','feature_wspomaganie-kierownicy__cat','param_uszkodzony__cat','feature_światła-led__cat','param_liczba-miejsc__cat','feature_łopatki-zmiany-biegów__cat','feature_regulowane-zawieszenie__cat']

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values
  model = model
  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [11]:
xgbparams = {
    'n_estimators': 50, 
    'max_depth': 5,
    'learning_rate': 0.3,
    'seed': 0,
    'random_state': 2020
}

model = xgb.XGBRegressor(**xgbparams)
run_model(model, feats)

[17:47:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:47:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:47:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:47:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:48:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-8573.002573351745, 100.36364939678465)

In [12]:
def check_params(params):
  print('Training with params: ')
  print(params)

  mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)
  return { 'loss': np.abs(mean_mae), 'status': STATUS_OK}

xgb_reg_params = {
    'learning_rate': hp.choice('learning_rate', np.arange(0.05, 0.31, 0.05)),
    'max_depth': hp.choice('max_depth', np.arange(5, 16, 1, dtype=int)),
    'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'objective': 'reg:squarederror', 
    'n_estimators': 100, 
    'seed': 0
}

#run
best = fmin(check_params, xgb_reg_params, algo=tpe.suggest, max_evals=3)

best

Training with params: 
{'colsample_bytree': 0.5, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.6000000000000001}
Training with params: 
{'colsample_bytree': 0.8, 'learning_rate': 0.15000000000000002, 'max_depth': 7, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9500000000000001}
Training with params: 
{'colsample_bytree': 0.9, 'learning_rate': 0.3, 'max_depth': 7, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.7000000000000001}
100%|██████████| 3/3 [03:04<00:00, 56.91s/it, best loss: 7571.820731850066]


{'colsample_bytree': 0.8,
 'learning_rate': 2,
 'max_depth': 2,
 'subsample': 0.9500000000000001}